In [1]:
%pip install requests folium polyline

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import folium
import polyline

In [3]:
# Definindo as chaves das APIs e as URLs base
openrouteservice_api_key = "5b3ce3597851110001cf6248f5c43779a8914cd39cc5548701fa43f4"
openweather_api_key = "d1a1abb0f31afc77d45ed70227666f91"

In [4]:
# Coordenadas das cidades de origem e destino
cidade_origem = "Curitiba"
cidade_destino = "Porto Alegre"

In [5]:
# Função para obter coordenadas e clima de uma cidade usando a API do OpenWeatherMap
def obter_coordenadas_clima(cidade, api_key):
    weather_url = f"http://api.openweathermap.org/data/2.5/weather?q={cidade},br&lang=pt&units=metric&appid={api_key}"
    response = requests.get(weather_url)
    data = response.json()

    weather = data["weather"][0]["description"]
    temperature = data["main"]["temp"]
    latitude = data["coord"]["lat"]
    longitude = data["coord"]["lon"]

    return (latitude, longitude), weather, temperature

In [6]:
origem_coords, origem_weather, origem_temp = obter_coordenadas_clima(
    cidade_origem, openweather_api_key
)

# Exibindo os dados do clima
print(f"Clima em {cidade_origem}: {origem_weather}")
print(
    f"Temperatura em {cidade_origem}: {origem_temp}°C"
)  # Convertendo de Kelvin para Celsius
print(
    f"Coordenadas de {cidade_origem}: Latitude {origem_coords[0]}, Longitude {origem_coords[1]}"
)

Clima em Curitiba: neblina
Temperatura em Curitiba: 11.48°C
Coordenadas de Curitiba: Latitude -25.504, Longitude -49.2908


In [7]:
destino_coords, destino_weather, destino_temp = obter_coordenadas_clima(
    cidade_destino, openweather_api_key
)

# Exibindo os dados do clima
print(f"Clima em {cidade_destino}: {destino_weather}")
print(
    f"Temperatura em {cidade_destino}: {destino_temp}°C"
)  # Convertendo de Kelvin para Celsius
print(
    f"Coordenadas de {cidade_destino}: Latitude {destino_coords[0]}, Longitude {destino_coords[1]}"
)

Clima em Porto Alegre: nuvens quebradas
Temperatura em Porto Alegre: 17.65°C
Coordenadas de Porto Alegre: Latitude -30.0331, Longitude -51.23


In [8]:
# URL para a API do OpenRouteService
route_url = "https://api.openrouteservice.org/v2/directions/driving-car"
route_headers = {
    "Authorization": openrouteservice_api_key,
    "Content-Type": "application/json",
}
route_payload = {
    "coordinates": [
        [origem_coords[1], origem_coords[0]],
        [destino_coords[1], destino_coords[0]],
    ],
    "instructions": False,
}

In [9]:
# Fazendo a requisição POST para a API do OpenRouteService
route_response = requests.post(route_url, json=route_payload, headers=route_headers)
route_data = route_response.json()

print(route_data)

{'bbox': [-51.230023, -30.033357, -48.595529, -25.504052], 'routes': [{'summary': {'distance': 741605.4, 'duration': 31592.2}, 'bbox': [-51.230023, -30.033357, -48.595529, -25.504052], 'geometry': 'hgtzCrdzkHz@KGy@CeADmAJeAPw@H]`@iAd@}@x@uALSnAyBtCaFfD{FVLz@`@bDrBpPbMhA~@lEbDbJpE`JzEvA|@zApAj@`@\\P~GlDh@Vz@Zz@Rn@T`Bv@`HxDxBjAxDjB|K~FtAr@|@Vv@XfFhCdBv@fA\\dEdArC|@|Bj@zKtC\\JdG~AbAPtEr@|@NhBVlDh@bHbA~Ad@pAXrUlD^D~Ev@p@Jp@V`@V\\\\Pf@DX?XAZGZIROXONSJWHUDW?WCi@SQMQUM[GYE]AaAz@oHTiBLoFlAiK`CsS|@yG`BeN|@yFnAsG|@eEfBaHhA_Ev@_CpB_GnAqDrDcJbQq^Zm@Xm@~KkUv@eBnDmH~GmNzJcTTm@jB{El@kBtA{ExAmGj@{Cd@sC`CaQpAgJ|QssAj@iErCuSdAeJr@eIjAcQXiEzEgt@^sF|@aNfIcoAHeBFwB?]?eBCeCQgEsB_U[gE{AwPqAsNQqBsB{UIsC?g@Dc@XsAPi@jH}Or@cAr@k@b@SzAm@hGcBxD_AfMyCfc@kK~Ao@r@a@j@c@p@s@dCcDpHqJv@u@tA_AvCcB`CqAzAs@~@[hASl@CrA?`@B`ANhFvAxIjC~NjEb@JfANpAFf@?xBSx@IbE]fOgA|WeC`JaA~T}BnB[`Ce@lD_Ali@iQh_@}LpIqCzu@yVpDy@`G_Ad@G|J{AhDi@xF{@nIqAnDi@lNqBrTiD~VsDnCi@dFq@xLqBpNyCb]kHbASb]iHdx@uPvOeDvf@iLtM}C~Be@nBYtBQ|@E~AClIBhBGn@GrC_@hAWn@Q

In [10]:
# Extraindo a rota usando a biblioteca polyline para decodificar
route_coords = polyline.decode(route_data["routes"][0]["geometry"])

# Calculando a caixa delimitadora (bounding box) da rota
min_lat = min(coord[0] for coord in route_coords)
max_lat = max(coord[0] for coord in route_coords)
min_lon = min(coord[1] for coord in route_coords)
max_lon = max(coord[1] for coord in route_coords)

# Calculando o centro da caixa delimitadora
center_lat = (min_lat + max_lat) / 2
center_lon = (min_lon + max_lon) / 2


# Criando o mapa com a rota
route_map = folium.Map(location=[(min_lat + max_lat) / 2, (min_lon + max_lon) / 2])

folium.PolyLine(locations=route_coords, color="blue", weight=2.5).add_to(route_map)
folium.Marker(
    location=origem_coords, popup=cidade_origem, icon=folium.Icon(color="green")
).add_to(route_map)
folium.Marker(
    location=destino_coords, popup=cidade_destino, icon=folium.Icon(color="red")
).add_to(route_map)

# Ajustando os limites do mapa para caber toda a rota
route_map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])

route_map